In [1]:
from sklearn.ensemble import VotingClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn import preprocessing
import pandas as pd

df_train = pd.read_csv("./stat679final/data/metadata/train_metadata.csv")
df_test = pd.read_csv("./stat679final/data/metadata/test_metadata.csv")
X_train, y_train = df_train.iloc[:, 0:6], df_train["class"]
# X, y = df.iloc[:, 0:6], df["class"]
X_test, y_test = df_test.iloc[:, 0:6], df_test["class"]

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Instantiate the individual classifiers
knn_clf = KNeighborsClassifier(n_neighbors=3)
tree_clf = DecisionTreeClassifier()
lr_clf = LogisticRegression(max_iter=3000)

In [2]:
# svm_clf = SVC(probability=True)  # Enable probability for soft voting

# Create a voting classifier
voting_clf_hard = VotingClassifier(
    estimators=[('knn', knn_clf), ('dt', tree_clf)],
    voting='hard'
)
voting_clf_soft = VotingClassifier(
    estimators=[('knn', knn_clf), ('dt', tree_clf)],
    voting='soft'
)

# Train the classifiers
voting_clf_hard.fit(X_train, y_train)
print("hard")
voting_clf_soft.fit(X_train, y_train)
print("soft")
# Make predictions and evaluate
y_pred_hard = voting_clf_hard.predict(X_test)
y_pred_soft = voting_clf_soft.predict(X_test)

print("Hard Voting Accuracy:", accuracy_score(y_test, y_pred_hard))
print("Soft Voting Accuracy:", accuracy_score(y_test, y_pred_soft))

hard
soft
Hard Voting Accuracy: 0.968598429921496
Soft Voting Accuracy: 0.9742487124356218


In [3]:
import numpy as np
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
from torchvision import transforms
from PIL import Image


class PyTorchClassifierWrapper:
    def __init__(self, model, device='cpu'):
        self.model = model
        self.model.to(device)
        self.device = device

    def predict_proba(self, dataloader):
        # Ensure the model is in evaluation mode
        self.model.eval()
        
        # List to hold all batch probabilities
        all_probabilities = []

        # Loop over all batches in the dataloader
        for X_batch in dataloader:
            images, label = X_batch
            # Transfer batch to the device (GPU or CPU)
            images = images.to(self.device)

            # Disable gradient computation
            with torch.no_grad():
                if images == 'unreadable':
                    outputs = tensor([[ 1/3, 1/3, 1/3]])
                else: 
                    outputs = self.model(images)

            # Apply softmax to compute probabilities
            probabilities = F.softmax(outputs, dim=1)
            
            # Move probabilities to CPU and convert to numpy
            probabilities = probabilities.cpu().numpy()
            
            # Append batch probabilities to list
            all_probabilities.append(probabilities)

        # Concatenate all batch probabilities into a single array
        all_probabilities = np.concatenate(all_probabilities, axis=0)
        
        return all_probabilities
    def predict(self,dataloader):
        all_predictions = []
        for X_batch in dataloader:
            images, label = X_batch
            # Transfer batch to the device (GPU or CPU)
            images = images.to(self.device)

            # Disable gradient computation
            with torch.no_grad():
                if images == 'unreadable':
                    outputs = tensor([[ 1/3, 1/3, 1/3]])
                else: 
                    outputs = self.model(images)
            max_indices = outputs.argmax(dim=-1)

            # Convert indices to one-hot encoded tensor
            predictions = F.one_hot(max_indices, num_classes=outputs.shape[-1]).cpu().numpy()

            # Move probabilities to CPU and convert to numpy
            
            # Append batch probabilities to list
            all_predictions.append(predictions)

        # Concatenate all batch probabilities into a single array
        all_predictions = np.concatenate(all_predictions, axis=0)
        
        return all_predictions

    
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Input channels = 3 (RGB), 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 13 * 13, 120)  # Adjusted for 64x64 input images
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)  # Output 3 classes: STAR, QSO, GALAXY

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 13 * 13)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


F:\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [4]:
model = SimpleCNN()
model.load_state_dict(torch.load('./stat679final/model/CNN/SimpleCNN_state_dict.pth'))
model.eval()  
print(model)


# Define the same transformation as during the training
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Example resize, adjust to your model's input
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the image
image = Image.open('./stat679final/data/GALAXY/GALAXY_1.jpg')
image = transform(image).unsqueeze(0)  # Add batch dimension
with torch.no_grad():  # Turn off gradients for inference
    output = model(image)
    # Assuming classification: get the predicted class (the index of the max log-probability)
    predicted_class = torch.argmax(output, dim=1)
    if predicted_class.item()+1 == 1:
      print("it is a galaxy")
    print(output)

SimpleCNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2704, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=3, bias=True)
)
it is a galaxy
tensor([[ 3.7507, -0.3346, -4.1034]])


In [5]:
model_spec = SimpleCNN()
model_spec.load_state_dict(torch.load('./stat679final/model/CNN/spec_state_dict.pth'))
model_spec.eval()  
print(model_spec)


# Define the same transformation as during the training
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Example resize, adjust to your model's input
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the image
image = Image.open('./data_spec/GALAXY_spec/GALAXY_1.jpg').convert('RGB')
image = transform(image).unsqueeze(0)  # Add batch dimension
with torch.no_grad():  # Turn off gradients for inference
    output = model_spec(image)
    # Assuming classification: get the predicted class (the index of the max log-probability)
    predicted_class = torch.argmax(output, dim=1)
    if predicted_class.item()+1 == 1:
        print("it is a galaxy")
    print(output)

SimpleCNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2704, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=3, bias=True)
)
it is a galaxy
tensor([[ 2.8297, -1.9385, -1.7206]])


In [6]:
def soft_voting(models, datasets):
    """
    对多个模型的预测概率进行平均，返回最终预测类别。
    :param models: 包含四个模型的列表
    :param datasets: 每个模型对应的测试数据集列表
    :return: 最终的预测类别数组
    """
    # 收集所有模型的预测概率
    proba_lists = [model.predict_proba(data) for model, data in zip(models, datasets)]

    # 计算平均预测概率
    avg_proba = np.mean(proba_lists, axis=0)

    # 选择平均概率最高的类别作为最终预测
    final_predictions = np.argmax(avg_proba, axis=1)
    return final_predictions


In [7]:
import numpy as np

def weighted_hard_voting(models, datasets, weights):
    """
    对多个模型的预测类别进行加权硬投票，返回最终预测类别。
    :param models: 包含四个模型的列表
    :param datasets: 每个模型对应的测试数据集列表
    :param weights: 每个模型在训练集上的准确率作为权重，列表形式
    :return: 最终的预测类别数组
    """
    # 收集所有模型的预测类别
    predictions = [model.predict(data) for model, data in zip(models, datasets)]
    
    # 转换预测为权重计票矩阵
    weighted_votes = np.zeros((datasets[3].shape[0], np.max(predictions) + 1))  # 假设类别从0开始

    # 累加每个模型的预测权重到对应类别
    for prediction, weight in zip(predictions, weights):
        for i, pred in enumerate(prediction):
            weighted_votes[i, pred] += weight

    # 选择权重计票最高的类别作为最终预测
    final_predictions = np.argmax(weighted_votes, axis=1)
    return final_predictions

In [8]:
pytorch_model1 = PyTorchClassifierWrapper(model, device='cpu')
pytorch_model2 = PyTorchClassifierWrapper(model_spec, device='cpu')

In [9]:
knn_clf.fit(X_train, y_train)
tree_clf.fit(X_train, y_train)
lr_clf.fit(X_train, y_train)

LogisticRegression(max_iter=3000)

In [10]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [12]:
test_image = pd.read_csv("./stat679final/data/rnk_test.csv")
#test_image

In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to ensure all images are the same size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize images
])

In [15]:
from PIL import Image
from torch.utils.data import Dataset
class CustomImageDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels  # Store the labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]  # Get the label for the current image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            try:
                image = self.transform(image)
                return image, label  # Return both the image and the label
            except IOError:
                return 'unreadable', lable
# Assuming 'class' column in image_paths_df contains labels
img_paths = test_image['image']
labels = test_image['class']  # Load labels from the dataframe

# Initialize dataset with paths and labels
custom_dataset = CustomImageDataset(img_paths, labels, transform=transform)
custom_data_loader = DataLoader(custom_dataset, batch_size=64, shuffle=True)

img_paths_spec = test_image['spec']
labels = test_image['class']  # Load labels from the dataframe

# Initialize dataset with paths and labels
custom_dataset_spec = CustomImageDataset(img_paths, labels, transform=transform)
custom_data_loader_spec = DataLoader(custom_dataset_spec, batch_size=64, shuffle=True)

In [16]:
# scikit-learn模型不需要wrapper
models = [pytorch_model1,pytorch_model2, knn_clf,tree_clf,lr_clf]
datasets = [custom_data_loader,custom_data_loader_spec,X_test,X_test,X_test]
# 执行软投票
final_predictions = soft_voting(models, datasets)

In [17]:
# weights = [0.918, 0.9914,0.8722,0.9159]
# # scikit-learn模型不需要wrapper
# models = [pytorch_model1,pytorch_model2, knn_clf, tree_clf]
# datasets = [custom_data_loader,custom_data_loader,X_test,X_test]
# # 执行软投票
# final_predictions = weighted_hard_voting(models, datasets,weights)

In [18]:
# # scikit-learn模型不需要wrapper
# models = [knn_clf, tree_clf,lr_clf]
# datasets = [X_test,X_test,X_test]
# # 执行软投票
# final_predictions = soft_voting(models, datasets)

In [19]:
(knn_clf.predict(X_test) == y_test).sum()/len(y_test)

0.967948397419871

In [20]:
(tree_clf.predict(X_test) == y_test).sum()/len(y_test)

0.9764488224411221

In [21]:
y_test = df_test["class"]
(y_test == final_predictions).sum()/len(y_test)

0.9770988549427472